In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, BatchNormalization, LeakyReLU, ReLU, Reshape
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
saved_model = tf.keras.models.load_model("gan_evaluation/gan_evaluation_model.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
latent_shape = 100
batch_size = 32
n_epochs=50

In [6]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.mnist.load_data()

# Find indices of labels 5 to 9
train_index = np.squeeze(np.argwhere(y_train_full>=5))
# test_index = np.squeeze(np.argwhere(y_test_full>=5))

(x_train, y_train) = (x_train_full[train_index], y_train_full[train_index])
# (x_test, y_test) = (x_test_full[test_index], y_test_full[test_index])

x_train_n = np.expand_dims(x_train/255, axis=-1).astype('float32')
# x_test_n = np.expand_dims(x_test/255, axis=-1).astype('float32')

num_classes = 5

# convert class vectors to binary class matrices
y_train_b = to_categorical(y_train-5, num_classes)
# y_test_b = to_categorical(y_test-5, num_classes)

dataset = tf.data.Dataset.from_tensor_slices(x_train_n)
dataset = dataset.shuffle(buffer_size=100).batch(batch_size, drop_remainder=True).prefetch(1)

In [7]:
# nb of steps
x_train_n.shape[0]/batch_size*n_epochs

45943.75

In [8]:
init = RandomNormal(stddev=0.02)
clip_value = 0.01

In [9]:
critic = tf.keras.Sequential(name="critic")
critic.add(Conv2D(64, (4, 4), strides=(2, 2), padding="same", kernel_initializer=init, input_shape=(28, 28, 1), name="c_conv_1"))
critic.add(BatchNormalization(name="c_batch_norm_1"))
critic.add(LeakyReLU(alpha=0.2, name="c_leaky_1"))
critic.add(Conv2D(64, (4, 4), strides=(2, 2), padding="same", kernel_initializer=init, name="c_conv_2"))
critic.add(BatchNormalization(name="c_batch_norm_2"))
critic.add(LeakyReLU(alpha=0.2, name="c_leaky_2"))
critic.add(Flatten(name="c_flatten"))
critic.add(Dense(128, name="c_dense_1"))
critic.add(Dense(1, name="c_dense_out"))
critic.add(LeakyReLU(alpha=0.2, name="c_leaky_out"))

In [10]:
critic.summary()

Model: "critic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c_conv_1 (Conv2D)            (None, 14, 14, 64)        1088      
_________________________________________________________________
c_batch_norm_1 (BatchNormali (None, 14, 14, 64)        256       
_________________________________________________________________
c_leaky_1 (LeakyReLU)        (None, 14, 14, 64)        0         
_________________________________________________________________
c_conv_2 (Conv2D)            (None, 7, 7, 64)          65600     
_________________________________________________________________
c_batch_norm_2 (BatchNormali (None, 7, 7, 64)          256       
_________________________________________________________________
c_leaky_2 (LeakyReLU)        (None, 7, 7, 64)          0         
_________________________________________________________________
c_flatten (Flatten)          (None, 3136)              0    

In [11]:
generator = tf.keras.Sequential(name="generator")
generator.add(Dense(100, input_shape=[latent_shape], name="g_dense_in"))
generator.add(ReLU(name="g_relu_1"))

generator.add(Dense(128*7*7, input_shape=[latent_shape], name="g_dense_2"))
generator.add(ReLU(name="g_relu_2"))
generator.add(Reshape((7, 7, 128)))

generator.add(Conv2DTranspose(64, (4, 4),strides=(2, 2), padding="same", kernel_initializer=init, name="g_convT_1"))
generator.add(BatchNormalization(name="g_batch_norm_1"))
generator.add(ReLU(name="g_relu_3"))

generator.add(Conv2DTranspose(1, (4, 4), strides=(2, 2), padding="same", kernel_initializer=init, activation="tanh", name="g_convT_2"))
generator.add(BatchNormalization(name="g_batch_norm_2"))
# generator.add(ReLU(name="g_relu_4"))
# generator.add(Conv2D(1, (7,7), activation="tanh", padding='same', name="g_conv_out"))

In [12]:
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_dense_in (Dense)           (None, 100)               10100     
_________________________________________________________________
g_relu_1 (ReLU)              (None, 100)               0         
_________________________________________________________________
g_dense_2 (Dense)            (None, 6272)              633472    
_________________________________________________________________
g_relu_2 (ReLU)              (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
g_convT_1 (Conv2DTranspose)  (None, 14, 14, 64)        131136    
_________________________________________________________________
g_batch_norm_1 (BatchNormali (None, 14, 14, 64)        25

In [13]:
gan = tf.keras.Sequential([generator, critic], name="gan")
gan.summary()

Model: "gan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator (Sequential)       (None, 28, 28, 1)         775993    
_________________________________________________________________
critic (Sequential)          (None, 1)                 468865    
Total params: 1,244,858
Trainable params: 1,244,472
Non-trainable params: 386
_________________________________________________________________


In [14]:
def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)

In [15]:
critic.compile(loss=wasserstein_loss, optimizer=RMSprop(lr=0.00005, clipvalue=clip_value))
# critic.trainable = False
gan.compile(loss=wasserstein_loss, optimizer=RMSprop(lr=0.00005, clipvalue=clip_value))

In [16]:
def train_gan(gan, dataset, batch_size, latent_shape, n_epochs=n_epochs, n_critic=5):
    c_hist, g_hist = list(), list()
    generator, critic = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            
            # phase 1 - training the critic
            c_tmp = list()
            for _ in range(n_critic):
                noise = tf.random.normal(shape=[batch_size, latent_shape])
                generated_images = generator(noise)
                X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
                y1 = tf.constant([[1.]] * batch_size + [[-1.]] * batch_size)
                critic.trainable = True
                c_loss = critic.train_on_batch(X_fake_and_real, y1)
                c_tmp.append(c_loss)
                                    
            c_hist.append(np.mean(c_tmp))
            
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, latent_shape])
            y2 = tf.constant([[1.]] * batch_size)
            critic.trainable = False
            g_loss = gan.train_on_batch(noise, y2)
            g_hist.append(g_loss)
        
        print('>%d, c=%.3f, g=%.3f' % (epoch, c_hist[-1], g_loss))
        
        # predictions
        y_prob = saved_model.predict(generated_images)
        for i in range(batch_size):
            label = np.argmax(y_prob[i,:])+5
            prob = np.max(y_prob[i,:])
            # PLOT
            plt.subplot(4, 8, 1 + i)
            plt.axis('off')
            plt.imshow(generated_images[i, :, :, 0], cmap='gray_r')
            plt.title("L={}\nP={:.2f}".format(label, prob), size=10)
            # save plot to file
        filename1 = 'eval_gan_plot_%04d.png' % (epoch+1)
        plt.savefig(os.path.join("gan_evaluation", filename1), dpi=200, size=(10, 10))
        plt.close()
        # save the generator model
        filename2 = os.path.join("gan_evaluation",'attack_gan_model_%04d.h5' % (epoch+1))
        gan.save(filename2)
        # save the critic model
        filename3 = os.path.join("gan_evaluation",'attack_critic_model_weights_%04d.h5' % (epoch+1))
        critic.save_weights(filename3)
        

In [17]:
train_gan(gan, dataset, batch_size, latent_shape)

>0, c=-28068.535, g=-4858.824
>1, c=-129914.562, g=-22779.910
>2, c=-338398.844, g=-58719.902
>3, c=-696579.250, g=-111467.375
>4, c=-1217413.000, g=-199865.469
>5, c=-1941423.625, g=-331694.188
>6, c=-2885706.750, g=-491547.500
>7, c=-4074969.250, g=-695947.625
>8, c=-5540040.500, g=-944504.500
>9, c=-7306177.000, g=-1237091.250
>10, c=-9401341.000, g=-1606751.000
>11, c=-11849793.000, g=-2027238.250
>12, c=-14679034.000, g=-2531039.000
>13, c=-17911180.000, g=-3102326.000
>14, c=-21569382.000, g=-3746009.000
>15, c=-25685532.000, g=-4465330.000
>16, c=-30286070.000, g=-5275254.500
>17, c=-35400972.000, g=-6169782.000
>18, c=-41054984.000, g=-7182955.000
>19, c=-47276940.000, g=-8232397.500
>20, c=-54096424.000, g=-9454419.000
>21, c=-61535796.000, g=-10804909.000
>22, c=-69621784.000, g=-12203502.000
>23, c=-78379824.000, g=-13813800.000
>24, c=-87835920.000, g=-15519111.000
>25, c=-98001744.000, g=-17293628.000
>26, c=-108908928.000, g=-19251932.000
>27, c=-120597440.000, g=-2131046